# 银行卡支付欺诈检测

在此部分，我们将查看信用卡欺诈检测数据集，并构建一个二元分类模型，该模型可以根据提供的历史数据，将交易识别为欺诈性交易或有效交易。在 [2016 年的一项研究中](https://nilsonreport.com/upload/content_promo/The_Nilson_Report_10-17-2016.pdf)，人们发现信用卡欺诈给全球带来了 200 亿美元以上的损失。准确检测欺诈行为是一个很活跃的研究领域。

<img src=notebook_ims/fraud_detection.png width=50% />

### 带标签数据

你可以从 [Kaggle](https://www.kaggle.com/mlg-ulb/creditcardfraud/data) 下载支付欺诈数据集（Dal Pozzolo 等人，2015 年）。这个数据集包含成千上万的信用卡交易特征和标签，每个交易都标记为欺诈性交易或有效交易。在此 notebook 中，我们将根据这些交易的特征训练一个模型，以便未来能够预测有风险或欺诈性交易。

### 二元分类

因为我们有真实标签可以参考，所以将采用**监督式学习**方法，并训练二元分类器将数据分成两种交易类别之一：欺诈性交易或有效交易。我们将用训练数据训练模型，并看看模型泛化到测试数据的效果。

该 notebook 将分成以下几个步骤：
* 加载和探索数据
* 将数据拆分为训练集和测试集
* 定义和训练一个二元分类器 LinearLearner
* 改进模型
* 评估和比较模型测试效果

### 改进模型

这个 notebook 侧重于改进模型，如[这篇 SageMaker 博文](https://aws.amazon.com/blogs/machine-learning/train-faster-more-flexible-models-with-amazon-sagemaker-linear-learner/)所述。我们将完成以下两项任务：

1. **优化模型的超参数**，并根据某个指标改进模型，例如提高召回率或精确率。
2. **解决类别不平衡性问题**，即一个类别的训练样本比另一个类别的样本多得多（在此示例中，有效交易样本比欺诈性交易样本多得多）。

---

首先，导入常规资源。

In [1]:
import io
import os
import matplotlib.pyplot as plt
import numpy as np 
import pandas as pd 

import boto3
import sagemaker
from sagemaker import get_execution_role

%matplotlib inline

我将在下个单元格中存储 **SageMaker 变量**：
* sagemaker_session：我们训练模型时将使用的 SageMaker 会话。
* bucket：存储数据时使用的默认 S3 存储桶的名称。
* role：定义数据和模型权限的 IAM 角色。

In [2]:
# sagemaker session, role
sagemaker_session = sagemaker.Session()
role = sagemaker.get_execution_role()

# S3 bucket name
bucket = sagemaker_session.default_bucket()


## 加载和探索数据

接下来，加载数据并解压缩 `creditcardfraud.zip` 文件中的数据。此目录下有一个 csv 文件 `creditcard.csv`，其中包含所有交易数据。

与之前的 notebook 一样，一定要看看数据的分布，这样我们才知道如何开发欺诈检测模型。我们想要知道：数据集中的数据点数量、特征数量和类型，以及每个类别（有效或欺诈性交易）的数据分布。

In [3]:
# only have to run once
!wget https://s3.amazonaws.com/video.udacity-data.com/topher/2019/January/5c534768_creditcardfraud/creditcardfraud.zip
!unzip creditcardfraud


--2020-04-18 02:48:17--  https://s3.amazonaws.com/video.udacity-data.com/topher/2019/January/5c534768_creditcardfraud/creditcardfraud.zip
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.216.177.133
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.216.177.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 69155632 (66M) [application/zip]
Saving to: ‘creditcardfraud.zip.1’

creditcardfraud.zip 100%[===================>]  65.95M  7.16MB/s    in 9.6s    

2020-04-18 02:48:27 (6.90 MB/s) - ‘creditcardfraud.zip.1’ saved [69155632/69155632]

Archive:  creditcardfraud.zip
replace creditcard.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: ^C


In [5]:
# read in the csv file
local_data = 'creditcard.csv'

# print out some data
transaction_df = pd.read_csv(local_data)
print('Data shape (rows, cols): ', transaction_df.shape)
print()
transaction_df.head()

Data shape (rows, cols):  (284807, 31)



,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0


### 练习：计算欺诈性数据的百分比

看看每个类别（有效和欺诈性）的交易数据分布。

请完成以下函数 `fraudulent_percentage`。统计每个类别的数据点数量，并计算欺诈性数据点的百分比。

In [6]:
# Calculate the fraction of data points that are fraudulent
def fraudulent_percentage(transaction_df):
    '''Calculate the fraction of all data points that have a 'Class' label of 1; fraudulent.
       :param transaction_df: Dataframe of all transaction data points; has a column 'Class'
       :return: A fractional percentage of fraudulent data points/all points
    '''
    
    # your code here
    faudPer = transaction_df["Class"].sum() / transaction_df.shape[0]
    
    return faudPer


通过调用函数并输出结果，测试代码。

In [7]:
# call the function to calculate the fraud percentage
fraud_percentage = fraudulent_percentage(transaction_df)

print('Fraudulent percentage = ', fraud_percentage)
print('Total # of fraudulent pts: ', fraud_percentage*transaction_df.shape[0])
print('Out of (total) pts: ', transaction_df.shape[0])


Fraudulent percentage =  0.001727485630620034
Total # of fraudulent pts:  492.0
Out of (total) pts:  284807


### 练习：将数据划分为训练集和测试集

在此示例中，我们想要评估欺诈分类器的性能；使用训练数据训练模型，然后使用在训练过程中没见过的测试数据测试模型。我们需要将数据划分为训练集和测试集。

请完成以下函数 `train_test_split`。此函数应该：
* 随机重排交易数据
* 根据参数 `train_frac` 将数据分成两部分
* 获取训练和测试特征及标签
* 返回元组：(train_features, train_labels), (test_features, test_labels)

In [59]:
# split into train/test
def train_test_split(transaction_df, train_frac= 0.7, seed=1):
    '''Shuffle the data and randomly split into train and test sets;
       separate the class labels (the column in transaction_df) from the features.
       :param df: Dataframe of all credit card transaction data
       :param train_frac: The decimal fraction of data that should be training data
       :param seed: Random seed for shuffling and reproducibility, default = 1
       :return: Two tuples (in order): (train_features, train_labels), (test_features, test_labels)
       '''
    tempSelection = transaction_df.sample(frac=train_frac, random_state=seed)
    
    # shuffle and split the data
    train_features = tempSelection.loc[:,tempSelection.columns != "Class"]
    train_labels = tempSelection.loc[:,tempSelection.columns == "Class"]
    
    indexSelect = transaction_df.index.isin(tempSelection.index)
    tempSelectionTest = transaction_df[~indexSelect]

    test_features = tempSelectionTest.loc[:,tempSelectionTest.columns != "Class"]
    test_labels = tempSelectionTest.loc[:,tempSelectionTest.columns == "Class"]
    
    return (train_features.to_numpy(), train_labels.to_numpy().flatten()), (test_features.to_numpy(), test_labels.to_numpy().flatten())


### 测试单元格

在以下单元格中，我创建了训练数据和测试数据，并检查结果是否合理。以下测试会测试上述函数是否将数据划分成规定数量的数据点，并且标签的确是类别标签 (0, 1)。

In [60]:
# get train/test data
(train_features, train_labels), (test_features, test_labels) = train_test_split(transaction_df, train_frac=0.7)

In [61]:
# manual test

# for a split of 0.7:0.3 there should be ~2.33x as many training as test pts
print('Training data pts: ', len(train_features))
print('Test data pts: ', len(test_features))
print()

# take a look at first item and see that it aligns with first row of data
print('First item: \n', train_features[0])
print('Label: ', train_labels[0])
print()

# test split
assert len(train_features) > 2.333*len(test_features), \
        'Unexpected number of train/test points for a train_frac=0.7'
# test labels
assert np.all(train_labels)== 0 or np.all(train_labels)== 1, \
        'Train labels should be 0s or 1s.'
assert np.all(test_labels)== 0 or np.all(test_labels)== 1, \
        'Test labels should be 0s or 1s.'
print('Tests passed!')

Training data pts:  199365
Test data pts:  85442

First item: 
 [ 1.19907000e+05 -6.11711999e-01 -7.69705324e-01 -1.49759145e-01
 -2.24876503e-01  2.02857736e+00 -2.01988711e+00  2.92491387e-01
 -5.23020325e-01  3.58468461e-01  7.00499612e-02 -8.54022784e-01
  5.47347360e-01  6.16448382e-01 -1.01785018e-01 -6.08491804e-01
 -2.88559430e-01 -6.06199260e-01 -9.00745518e-01 -2.01311157e-01
 -1.96039343e-01 -7.52077614e-02  4.55360454e-02  3.80739375e-01
  2.34403159e-02 -2.22068576e+00 -2.01145578e-01  6.65013699e-02
  2.21179560e-01  1.79000000e+00]
Label:  0

Tests passed!


---
# 建模

上传训练数据后，下面定义和训练模型。

在此 notebook 中，你将定义和训练 SageMaker 的内置算法 [LinearLearner](https://sagemaker.readthedocs.io/en/stable/linear_learner.html)。 

LinearLearner 有两个主要用途：
1. 完成回归任务，即用一条线拟合一些数据点，你希望模型能根据某些数据点生成预测输出值（例如根据平方面积预测房价）
2. 用于二元分类，有一条线能区分两类数据并有效地输出标签；数据点在这条线上方的类别标签为 1，数据点在这条线上或下面的类别标签为 0。

<img src='notebook_ims/linear_separator.png' width=50% />

在此 notebook 中，我们将用于第二种情形，并训练 LinearLearner 将数据分成两个类别：有效或欺诈性数据。

### 练习：创建 LinearLearner 评估器

你已经练习过在 SageMaker 中实例化内置模型。所有评估器都需要传入一些构造函数参数。看看你能否仅参考 [LinearLearner 文档](https://sagemaker.readthedocs.io/en/stable/linear_learner.html)实例化一个 LinearLearner 评估器。此评估器有很多参数，但是并非必须都要指定。我的建议是从简单的模型开始，并尽量利用默认值。稍后，我们将讨论一些具体的超参数及其用例。

#### 实例类型

建议使用免费套餐里提供的实例“ml.c4.xlarge”训练模型，并使用“ml.t2.medium”部署模型。

In [51]:
session = sagemaker.Session()
role = get_execution_role()

In [52]:
# import LinearLearner
from sagemaker import LinearLearner

# instantiate LinearLearner
LinearLearner = LinearLearner(role=role,
                             train_instance_count=1,
                             train_instance_type="ml.c4.xlarge",
                             predictor_type ="binary_classifier")

### 练习：将数据转换成 RecordSet 格式

接下来，将训练特征和标签转换成浮点值 numpy 数组，为内置模型准备好数据。然后使用 [record_set 函数](https://sagemaker.readthedocs.io/en/stable/linear_learner.html#sagemaker.LinearLearner.record_set)将数据变成 RecordSet，并为训练做好准备。

In [65]:
train_features = train_features.astype('float32')
# create RecordSet of training data
formatted_train_data = LinearLearner.record_set(train_features, labels=train_labels.flatten().astype('float32'))

### 练习：训练评估器

实例化评估器后，通过调用 `.fit()` 训练该评估器，并传入特殊格式的训练数据。

In [ ]:
%%time 
# train the estimator on formatted training data
LinearLearner.fit(formatted_train_data)

2020-04-18 03:41:05 Starting - Starting the training job...
2020-04-18 03:41:07 Starting - Launching requested ML instances........
2020-04-18 03:43:33 Downloading - Downloading input data
2020-04-18 03:43:33 Training - Downloading the training image...
2020-04-18 03:43:56 Training - Training image download completed. Training in progress.Docker entrypoint called with argument(s): train
Running default environment configuration script
[04/18/2020 03:43:59 INFO 140327549495104] Reading default configuration from /opt/amazon/lib/python2.7/site-packages/algorithm/resources/default-input.json: {u'loss_insensitivity': u'0.01', u'epochs': u'15', u'feature_dim': u'auto', u'init_bias': u'0.0', u'lr_scheduler_factor': u'auto', u'num_calibration_samples': u'10000000', u'accuracy_top_k': u'3', u'_num_kv_servers': u'auto', u'use_bias': u'true', u'num_point_for_scaler': u'10000', u'_log_level': u'info', u'quantile': u'0.5', u'bias_lr_mult': u'auto', u'lr_scheduler_step': u'auto', u'init_method': u'

[2020-04-18 03:44:11.845] [tensorio] [info] epoch_stats={"data_pipeline": "/opt/ml/input/data/train", "epoch": 5, "duration": 6459, "num_examples": 200, "num_bytes": 33493320}
#metrics {"Metrics": {"train_binary_classification_cross_entropy_objective": {"count": 1, "max": 0.01697396398668912, "sum": 0.01697396398668912, "min": 0.01697396398668912}}, "EndTime": 1587181451.845615, "Dimensions": {"model": 0, "Host": "algo-1", "Operation": "training", "Algorithm": "Linear Learner", "epoch": 1}, "StartTime": 1587181451.84551}

#metrics {"Metrics": {"train_binary_classification_cross_entropy_objective": {"count": 1, "max": 0.014988560247660881, "sum": 0.014988560247660881, "min": 0.014988560247660881}}, "EndTime": 1587181451.845705, "Dimensions": {"model": 1, "Host": "algo-1", "Operation": "training", "Algorithm": "Linear Learner", "epoch": 1}, "StartTime": 1587181451.845684}

#metrics {"Metrics": {"train_binary_classification_cross_entropy_objective": {"count": 1, "max": 0.01696843108459932

[2020-04-18 03:44:24.141] [tensorio] [info] epoch_stats={"data_pipeline": "/opt/ml/input/data/train", "epoch": 9, "duration": 6065, "num_examples": 200, "num_bytes": 33493320}
#metrics {"Metrics": {"train_binary_classification_cross_entropy_objective": {"count": 1, "max": 0.008034130657138537, "sum": 0.008034130657138537, "min": 0.008034130657138537}}, "EndTime": 1587181464.141676, "Dimensions": {"model": 0, "Host": "algo-1", "Operation": "training", "Algorithm": "Linear Learner", "epoch": 3}, "StartTime": 1587181464.141567}

#metrics {"Metrics": {"train_binary_classification_cross_entropy_objective": {"count": 1, "max": 0.006699231958868516, "sum": 0.006699231958868516, "min": 0.006699231958868516}}, "EndTime": 1587181464.141783, "Dimensions": {"model": 1, "Host": "algo-1", "Operation": "training", "Algorithm": "Linear Learner", "epoch": 3}, "StartTime": 1587181464.141761}

#metrics {"Metrics": {"train_binary_classification_cross_entropy_objective": {"count": 1, "max": 0.0080317853203

[2020-04-18 03:44:30.393] [tensorio] [info] epoch_stats={"data_pipeline": "/opt/ml/input/data/train", "epoch": 11, "duration": 6244, "num_examples": 200, "num_bytes": 33493320}
#metrics {"Metrics": {"train_binary_classification_cross_entropy_objective": {"count": 1, "max": 0.0069169620389315355, "sum": 0.0069169620389315355, "min": 0.0069169620389315355}}, "EndTime": 1587181470.393385, "Dimensions": {"model": 0, "Host": "algo-1", "Operation": "training", "Algorithm": "Linear Learner", "epoch": 4}, "StartTime": 1587181470.393277}

#metrics {"Metrics": {"train_binary_classification_cross_entropy_objective": {"count": 1, "max": 0.00574524320429893, "sum": 0.00574524320429893, "min": 0.00574524320429893}}, "EndTime": 1587181470.393485, "Dimensions": {"model": 1, "Host": "algo-1", "Operation": "training", "Algorithm": "Linear Learner", "epoch": 4}, "StartTime": 1587181470.393465}

#metrics {"Metrics": {"train_binary_classification_cross_entropy_objective": {"count": 1, "max": 0.006915968100

[2020-04-18 03:44:42.734] [tensorio] [info] epoch_stats={"data_pipeline": "/opt/ml/input/data/train", "epoch": 15, "duration": 6323, "num_examples": 200, "num_bytes": 33493320}
#metrics {"Metrics": {"train_binary_classification_cross_entropy_objective": {"count": 1, "max": 0.005869037068668922, "sum": 0.005869037068668922, "min": 0.005869037068668922}}, "EndTime": 1587181482.734304, "Dimensions": {"model": 0, "Host": "algo-1", "Operation": "training", "Algorithm": "Linear Learner", "epoch": 6}, "StartTime": 1587181482.734205}

#metrics {"Metrics": {"train_binary_classification_cross_entropy_objective": {"count": 1, "max": 0.004911643637484642, "sum": 0.004911643637484642, "min": 0.004911643637484642}}, "EndTime": 1587181482.734399, "Dimensions": {"model": 1, "Host": "algo-1", "Operation": "training", "Algorithm": "Linear Learner", "epoch": 6}, "StartTime": 1587181482.734379}

#metrics {"Metrics": {"train_binary_classification_cross_entropy_objective": {"count": 1, "max": 0.005869176341

[2020-04-18 03:44:54.711] [tensorio] [info] epoch_stats={"data_pipeline": "/opt/ml/input/data/train", "epoch": 19, "duration": 5837, "num_examples": 200, "num_bytes": 33493320}
#metrics {"Metrics": {"train_binary_classification_cross_entropy_objective": {"count": 1, "max": 0.005391925841719661, "sum": 0.005391925841719661, "min": 0.005391925841719661}}, "EndTime": 1587181494.712124, "Dimensions": {"model": 0, "Host": "algo-1", "Operation": "training", "Algorithm": "Linear Learner", "epoch": 8}, "StartTime": 1587181494.712026}

#metrics {"Metrics": {"train_binary_classification_cross_entropy_objective": {"count": 1, "max": 0.0045703682456184275, "sum": 0.0045703682456184275, "min": 0.0045703682456184275}}, "EndTime": 1587181494.712222, "Dimensions": {"model": 1, "Host": "algo-1", "Operation": "training", "Algorithm": "Linear Learner", "epoch": 8}, "StartTime": 1587181494.712201}

#metrics {"Metrics": {"train_binary_classification_cross_entropy_objective": {"count": 1, "max": 0.005392487

[2020-04-18 03:45:05.834] [tensorio] [info] epoch_stats={"data_pipeline": "/opt/ml/input/data/train", "epoch": 23, "duration": 5535, "num_examples": 200, "num_bytes": 33493320}
#metrics {"Metrics": {"train_binary_classification_cross_entropy_objective": {"count": 1, "max": 0.005129939455482828, "sum": 0.005129939455482828, "min": 0.005129939455482828}}, "EndTime": 1587181505.834546, "Dimensions": {"model": 0, "Host": "algo-1", "Operation": "training", "Algorithm": "Linear Learner", "epoch": 10}, "StartTime": 1587181505.834435}

#metrics {"Metrics": {"train_binary_classification_cross_entropy_objective": {"count": 1, "max": 0.004400380541631325, "sum": 0.004400380541631325, "min": 0.004400380541631325}}, "EndTime": 1587181505.834656, "Dimensions": {"model": 1, "Host": "algo-1", "Operation": "training", "Algorithm": "Linear Learner", "epoch": 10}, "StartTime": 1587181505.834632}

#metrics {"Metrics": {"train_binary_classification_cross_entropy_objective": {"count": 1, "max": 0.0051306958

[2020-04-18 03:45:11.500] [tensorio] [info] epoch_stats={"data_pipeline": "/opt/ml/input/data/train", "epoch": 25, "duration": 5660, "num_examples": 200, "num_bytes": 33493320}
#metrics {"Metrics": {"train_binary_classification_cross_entropy_objective": {"count": 1, "max": 0.0050408838328404645, "sum": 0.0050408838328404645, "min": 0.0050408838328404645}}, "EndTime": 1587181511.501079, "Dimensions": {"model": 0, "Host": "algo-1", "Operation": "training", "Algorithm": "Linear Learner", "epoch": 11}, "StartTime": 1587181511.500969}

#metrics {"Metrics": {"train_binary_classification_cross_entropy_objective": {"count": 1, "max": 0.004347342520802464, "sum": 0.004347342520802464, "min": 0.004347342520802464}}, "EndTime": 1587181511.50117, "Dimensions": {"model": 1, "Host": "algo-1", "Operation": "training", "Algorithm": "Linear Learner", "epoch": 11}, "StartTime": 1587181511.501151}

#metrics {"Metrics": {"train_binary_classification_cross_entropy_objective": {"count": 1, "max": 0.00504167


2020-04-18 03:45:32 Uploading - Uploading generated training model[2020-04-18 03:45:23.686] [tensorio] [info] epoch_stats={"data_pipeline": "/opt/ml/input/data/train", "epoch": 29, "duration": 5915, "num_examples": 200, "num_bytes": 33493320}
#metrics {"Metrics": {"train_binary_classification_cross_entropy_objective": {"count": 1, "max": 0.004912319385825689, "sum": 0.004912319385825689, "min": 0.004912319385825689}}, "EndTime": 1587181523.686267, "Dimensions": {"model": 0, "Host": "algo-1", "Operation": "training", "Algorithm": "Linear Learner", "epoch": 13}, "StartTime": 1587181523.686158}

#metrics {"Metrics": {"train_binary_classification_cross_entropy_objective": {"count": 1, "max": 0.004274392560199278, "sum": 0.004274392560199278, "min": 0.004274392560199278}}, "EndTime": 1587181523.686377, "Dimensions": {"model": 1, "Host": "algo-1", "Operation": "training", "Algorithm": "Linear Learner", "epoch": 13}, "StartTime": 1587181523.686355}

#metrics {"Metrics": {"train_binary_classi


2020-04-18 03:45:40 Completed - Training job completed
Training seconds: 148
Billable seconds: 148
CPU times: user 779 ms, sys: 0 ns, total: 779 ms
Wall time: 5min 12s


### 练习：部署训练过的模型

部署模型以创建预测器。我们将使用该预测器对测试数据进行预测并评估模型。

In [67]:
%%time 
# deploy and create a predictor
linear_predictor = LinearLearner.deploy(initial_instance_count=1,
                                       instance_type='ml.t2.medium')

---------------!CPU times: user 277 ms, sys: 0 ns, total: 277 ms
Wall time: 7min 31s


---
# 评估模型

部署模型后，可以评估模型在测试数据上的效果。

根据[预测器文档](https://sagemaker.readthedocs.io/en/stable/linear_learner.html#sagemaker.LinearLearnerPredictor)，此预测器要求传入 `ndarray` 格式的输入特征，并返回记录列表。
> "预测存储在 `Record.label` 字段的  "predicted_label" 键中。"

先测试下模型在一个测试数据点上的效果，看看生成的列表。

In [68]:
# test one prediction
test_x_np = test_features.astype('float32')
result = linear_predictor.predict(test_x_np[0])

print(result)

[label {
  key: "predicted_label"
  value {
    float32_tensor {
      values: 0.0
    }
  }
}
label {
  key: "score"
  value {
    float32_tensor {
      values: 0.0006405849126167595
    }
  }
}
]


### 评估辅助函数


以下函数的参数包括部署的预测器、测试特征和标签，并返回指标字典；计算真假正例和真假负例，以及召回率、精确率和准确率。

In [69]:
# code to evaluate the endpoint on test data
# returns a variety of model metrics
def evaluate(predictor, test_features, test_labels, verbose=True):
    """
    Evaluate a model on a test set given the prediction endpoint.  
    Return binary classification metrics.
    :param predictor: A prediction endpoint
    :param test_features: Test features
    :param test_labels: Class labels for test data
    :param verbose: If True, prints a table of all performance metrics
    :return: A dictionary of performance metrics.
    """
    
    # We have a lot of test data, so we'll split it into batches of 100
    # split the test data set into batches and evaluate using prediction endpoint    
    prediction_batches = [predictor.predict(batch) for batch in np.array_split(test_features, 100)]
    
    # LinearLearner produces a `predicted_label` for each data point in a batch
    # get the 'predicted_label' for every point in a batch
    test_preds = np.concatenate([np.array([x.label['predicted_label'].float32_tensor.values[0] for x in batch]) 
                                 for batch in prediction_batches])
    
    # calculate true positives, false positives, true negatives, false negatives
    tp = np.logical_and(test_labels, test_preds).sum()
    fp = np.logical_and(1-test_labels, test_preds).sum()
    tn = np.logical_and(1-test_labels, 1-test_preds).sum()
    fn = np.logical_and(test_labels, 1-test_preds).sum()
    
    # calculate binary classification metrics
    recall = tp / (tp + fn)
    precision = tp / (tp + fp)
    accuracy = (tp + tn) / (tp + fp + tn + fn)
    
    # printing a table of metrics
    if verbose:
        print(pd.crosstab(test_labels, test_preds, rownames=['actual (row)'], colnames=['prediction (col)']))
        print("\n{:<11} {:.3f}".format('Recall:', recall))
        print("{:<11} {:.3f}".format('Precision:', precision))
        print("{:<11} {:.3f}".format('Accuracy:', accuracy))
        print()
        
    return {'TP': tp, 'FP': fp, 'FN': fn, 'TN': tn, 
            'Precision': precision, 'Recall': recall, 'Accuracy': accuracy}


### 测试结果

以下单元格将运行 `evaluate` 函数。

代码假设你已经在之前运行的单元格中定义了 `predictor`、`test_features` 和 `test_labels`。

In [70]:
print('Metrics for simple, LinearLearner.\n')

# get metrics for linear predictor
metrics = evaluate(linear_predictor, 
                   test_features.astype('float32'), 
                   test_labels, 
                   verbose=True) # verbose means we'll print out the metrics


Metrics for simple, LinearLearner.

prediction (col)    0.0  1.0
actual (row)                
0                 85268   33
1                    32  109

Recall:     0.773
Precision:  0.768
Accuracy:   0.999



## 删除端点

我添加了一个便利函数，在使用完端点后，可以用它删除预测端点。评估完模型后，你应该删除模型端点。

In [71]:
# Deletes a precictor.endpoint
def delete_endpoint(predictor):
        try:
            boto3.client('sagemaker').delete_endpoint(EndpointName=predictor.endpoint)
            print('Deleted {}'.format(predictor.endpoint))
        except:
            print('Already deleted: {}'.format(predictor.endpoint))

In [72]:
# delete the predictor endpoint 
delete_endpoint(linear_predictor)

Deleted linear-learner-2020-04-18-03-41-05-401


---

# 改进模型

默认 LinearLearner 的准确率很高，但是依然将某些欺诈性数据点和有效数据点分类错了。将 30 多个数据点分类成假负例（错误标记的欺诈性交易），并且将 30 来个数据点分类成假正例（错误标记的有效交易）。我们思考下在训练过程中，什么会导致这种行为，该如何改进。

**1. 模型优化**
* 假设我们要为银行系统设计应用，那么用户肯定不希望有任何有效交易被归类为欺诈性交易。我们希望尽量减少**假正例**（0 被分类为 1）。
* 另一方面，银行经理要求应用能够发现几乎所有的欺诈性交易，即使假正例的数量很高，那么我们希望尽量减少**假负例**。
* 要根据特定的产品需求和目标训练模型，我们不能只提高准确率。我们需要优化有助于降低假正例或假负例数量的指标。

<img src='notebook_ims/precision_recall.png' width=40% />
     
在此 notebook 中，我们将查看优化模型和做出优化决策的不同情形。

**2. 不平衡的训练数据**
* 在此 notebook 的开头，我们看到只有 0.17% 的训练数据被标记成欺诈性交易。所以即使模型将**所有**数据标记为有效交易，准确率依然很高。
* 这样可能会导致过拟合有效数据，其中包含一些**假负例**；即欺诈性数据 (1) 被错误地归类为有效交易 (0)。

我们按顺序解决这些问题；首先，调节模型并在训练过程中根据特定指标优化模型；其次，解决训练集中的类别不平衡性问题。


## 改进模型：模型优化

根据特定指标进行优化称为**模型优化**，SageMaker 提供了多种自动优化模型的方式。


### 创建 LinearLearner 并提高精确率

**情形：**
* 银行请你构建一个检测欺诈性交易的模型，并且要求准确率约为 85%。

所以我们需要构建一个有很多真正例并且尽量减少假负例的模型。这种模型需要很高的**召回率**：真正例/（真正例 + 假负例）。

为了提高特定的指标，我们可以使用 LinearLearner 参数 `binary_classifier_model_selection_criteria`，它是训练数据集的模型评估标准。要了解该参数，请参阅此 [LinearLearner 文档](https://sagemaker.readthedocs.io/en/stable/linear_learner.html#sagemaker.LinearLearner)。我们还需要进一步指定到底要达到什么样的值；要详细了解此参数，请点击[此处](https://docs.aws.amazon.com/sagemaker/latest/dg/ll_hyperparameters.html)。

该参数将假设模型在训练集上的性能与在测试集上的性能相差约 5%。例如，如果召回率约为 85%，我将召回率提高到 90%。

In [74]:
prefix = 'creditcard'
output_path = 's3://{}/{}'.format(bucket, prefix)

In [76]:
# instantiate a LinearLearner
# import LinearLearner
from sagemaker import LinearLearner

# tune the model for a higher recall
linear_recall = LinearLearner(role=role,
                              train_instance_count=1, 
                              train_instance_type='ml.c4.xlarge',
                              predictor_type='binary_classifier',
                              output_path=output_path,
                              sagemaker_session=sagemaker_session,
                              epochs=15,
                              binary_classifier_model_selection_criteria='precision_at_target_recall', # target recall
                              target_recall=0.9) # 90% recall


### 训练优化过的评估器

用新的优化评估器拟合特殊格式的训练数据。

In [77]:
%%time 
# train the estimator on formatted training data
linear_recall.fit(formatted_train_data)

2020-04-18 04:35:18 Starting - Starting the training job...
2020-04-18 04:35:20 Starting - Launching requested ML instances......
2020-04-18 04:36:22 Starting - Preparing the instances for training...
2020-04-18 04:37:19 Downloading - Downloading input data...
2020-04-18 04:37:41 Training - Downloading the training image..Docker entrypoint called with argument(s): train
Running default environment configuration script
[04/18/2020 04:37:57 INFO 139971922900800] Reading default configuration from /opt/amazon/lib/python2.7/site-packages/algorithm/resources/default-input.json: {u'loss_insensitivity': u'0.01', u'epochs': u'15', u'feature_dim': u'auto', u'init_bias': u'0.0', u'lr_scheduler_factor': u'auto', u'num_calibration_samples': u'10000000', u'accuracy_top_k': u'3', u'_num_kv_servers': u'auto', u'use_bias': u'true', u'num_point_for_scaler': u'10000', u'_log_level': u'info', u'quantile': u'0.5', u'bias_lr_mult': u'auto', u'lr_scheduler_step': u'auto', u'init_method': u'uniform', u'init_

[2020-04-18 04:38:16.928] [tensorio] [info] epoch_stats={"data_pipeline": "/opt/ml/input/data/train", "epoch": 7, "duration": 6148, "num_examples": 200, "num_bytes": 33493320}
#metrics {"Metrics": {"train_binary_classification_cross_entropy_objective": {"count": 1, "max": 0.010330928617985404, "sum": 0.010330928617985404, "min": 0.010330928617985404}}, "EndTime": 1587184696.928248, "Dimensions": {"model": 0, "Host": "algo-1", "Operation": "training", "Algorithm": "Linear Learner", "epoch": 2}, "StartTime": 1587184696.928156}

#metrics {"Metrics": {"train_binary_classification_cross_entropy_objective": {"count": 1, "max": 0.008765228896883864, "sum": 0.008765228896883864, "min": 0.008765228896883864}}, "EndTime": 1587184696.928334, "Dimensions": {"model": 1, "Host": "algo-1", "Operation": "training", "Algorithm": "Linear Learner", "epoch": 2}, "StartTime": 1587184696.928313}

#metrics {"Metrics": {"train_binary_classification_cross_entropy_objective": {"count": 1, "max": 0.0103271253791

[2020-04-18 04:38:29.002] [tensorio] [info] epoch_stats={"data_pipeline": "/opt/ml/input/data/train", "epoch": 11, "duration": 6269, "num_examples": 200, "num_bytes": 33493320}
#metrics {"Metrics": {"train_binary_classification_cross_entropy_objective": {"count": 1, "max": 0.0069169620389315355, "sum": 0.0069169620389315355, "min": 0.0069169620389315355}}, "EndTime": 1587184709.002607, "Dimensions": {"model": 0, "Host": "algo-1", "Operation": "training", "Algorithm": "Linear Learner", "epoch": 4}, "StartTime": 1587184709.002511}

#metrics {"Metrics": {"train_binary_classification_cross_entropy_objective": {"count": 1, "max": 0.00574524320429893, "sum": 0.00574524320429893, "min": 0.00574524320429893}}, "EndTime": 1587184709.002688, "Dimensions": {"model": 1, "Host": "algo-1", "Operation": "training", "Algorithm": "Linear Learner", "epoch": 4}, "StartTime": 1587184709.002674}

#metrics {"Metrics": {"train_binary_classification_cross_entropy_objective": {"count": 1, "max": 0.006915968100

[2020-04-18 04:38:34.388] [tensorio] [info] epoch_stats={"data_pipeline": "/opt/ml/input/data/train", "epoch": 13, "duration": 5379, "num_examples": 200, "num_bytes": 33493320}
#metrics {"Metrics": {"train_binary_classification_cross_entropy_objective": {"count": 1, "max": 0.006276343655945668, "sum": 0.006276343655945668, "min": 0.006276343655945668}}, "EndTime": 1587184714.388651, "Dimensions": {"model": 0, "Host": "algo-1", "Operation": "training", "Algorithm": "Linear Learner", "epoch": 5}, "StartTime": 1587184714.388556}

#metrics {"Metrics": {"train_binary_classification_cross_entropy_objective": {"count": 1, "max": 0.005225318925464573, "sum": 0.005225318925464573, "min": 0.005225318925464573}}, "EndTime": 1587184714.388741, "Dimensions": {"model": 1, "Host": "algo-1", "Operation": "training", "Algorithm": "Linear Learner", "epoch": 5}, "StartTime": 1587184714.38872}

#metrics {"Metrics": {"train_binary_classification_cross_entropy_objective": {"count": 1, "max": 0.0062760830524

[2020-04-18 04:38:46.301] [tensorio] [info] epoch_stats={"data_pipeline": "/opt/ml/input/data/train", "epoch": 17, "duration": 6136, "num_examples": 200, "num_bytes": 33493320}
#metrics {"Metrics": {"train_binary_classification_cross_entropy_objective": {"count": 1, "max": 0.005591161470317361, "sum": 0.005591161470317361, "min": 0.005591161470317361}}, "EndTime": 1587184726.301781, "Dimensions": {"model": 0, "Host": "algo-1", "Operation": "training", "Algorithm": "Linear Learner", "epoch": 7}, "StartTime": 1587184726.301684}

#metrics {"Metrics": {"train_binary_classification_cross_entropy_objective": {"count": 1, "max": 0.004708522209570036, "sum": 0.004708522209570036, "min": 0.004708522209570036}}, "EndTime": 1587184726.301872, "Dimensions": {"model": 1, "Host": "algo-1", "Operation": "training", "Algorithm": "Linear Learner", "epoch": 7}, "StartTime": 1587184726.301852}

#metrics {"Metrics": {"train_binary_classification_cross_entropy_objective": {"count": 1, "max": 0.005591602163

[2020-04-18 04:38:58.660] [tensorio] [info] epoch_stats={"data_pipeline": "/opt/ml/input/data/train", "epoch": 21, "duration": 6373, "num_examples": 200, "num_bytes": 33493320}
#metrics {"Metrics": {"train_binary_classification_cross_entropy_objective": {"count": 1, "max": 0.005243560808387833, "sum": 0.005243560808387833, "min": 0.005243560808387833}}, "EndTime": 1587184738.660698, "Dimensions": {"model": 0, "Host": "algo-1", "Operation": "training", "Algorithm": "Linear Learner", "epoch": 9}, "StartTime": 1587184738.660572}

#metrics {"Metrics": {"train_binary_classification_cross_entropy_objective": {"count": 1, "max": 0.004472267618430919, "sum": 0.004472267618430919, "min": 0.004472267618430919}}, "EndTime": 1587184738.660815, "Dimensions": {"model": 1, "Host": "algo-1", "Operation": "training", "Algorithm": "Linear Learner", "epoch": 9}, "StartTime": 1587184738.660792}

#metrics {"Metrics": {"train_binary_classification_cross_entropy_objective": {"count": 1, "max": 0.005244257314

[2020-04-18 04:39:04.509] [tensorio] [info] epoch_stats={"data_pipeline": "/opt/ml/input/data/train", "epoch": 23, "duration": 5841, "num_examples": 200, "num_bytes": 33493320}
#metrics {"Metrics": {"train_binary_classification_cross_entropy_objective": {"count": 1, "max": 0.005129939455482828, "sum": 0.005129939455482828, "min": 0.005129939455482828}}, "EndTime": 1587184744.50973, "Dimensions": {"model": 0, "Host": "algo-1", "Operation": "training", "Algorithm": "Linear Learner", "epoch": 10}, "StartTime": 1587184744.509634}

#metrics {"Metrics": {"train_binary_classification_cross_entropy_objective": {"count": 1, "max": 0.004400380541631325, "sum": 0.004400380541631325, "min": 0.004400380541631325}}, "EndTime": 1587184744.509818, "Dimensions": {"model": 1, "Host": "algo-1", "Operation": "training", "Algorithm": "Linear Learner", "epoch": 10}, "StartTime": 1587184744.509801}

#metrics {"Metrics": {"train_binary_classification_cross_entropy_objective": {"count": 1, "max": 0.00513069580

[2020-04-18 04:39:16.309] [tensorio] [info] epoch_stats={"data_pipeline": "/opt/ml/input/data/train", "epoch": 27, "duration": 5538, "num_examples": 200, "num_bytes": 33493320}
#metrics {"Metrics": {"train_binary_classification_cross_entropy_objective": {"count": 1, "max": 0.004969843264201179, "sum": 0.004969843264201179, "min": 0.004969843264201179}}, "EndTime": 1587184756.309681, "Dimensions": {"model": 0, "Host": "algo-1", "Operation": "training", "Algorithm": "Linear Learner", "epoch": 12}, "StartTime": 1587184756.309582}

#metrics {"Metrics": {"train_binary_classification_cross_entropy_objective": {"count": 1, "max": 0.004306342939935138, "sum": 0.004306342939935138, "min": 0.004306342939935138}}, "EndTime": 1587184756.309773, "Dimensions": {"model": 1, "Host": "algo-1", "Operation": "training", "Algorithm": "Linear Learner", "epoch": 12}, "StartTime": 1587184756.309756}

#metrics {"Metrics": {"train_binary_classification_cross_entropy_objective": {"count": 1, "max": 0.0049706502

[2020-04-18 04:39:28.394] [tensorio] [info] epoch_stats={"data_pipeline": "/opt/ml/input/data/train", "epoch": 31, "duration": 6184, "num_examples": 200, "num_bytes": 33493320}
#metrics {"Metrics": {"train_binary_classification_cross_entropy_objective": {"count": 1, "max": 0.0048651198454238665, "sum": 0.0048651198454238665, "min": 0.0048651198454238665}}, "EndTime": 1587184768.394769, "Dimensions": {"model": 0, "Host": "algo-1", "Operation": "training", "Algorithm": "Linear Learner", "epoch": 14}, "StartTime": 1587184768.394684}

#metrics {"Metrics": {"train_binary_classification_cross_entropy_objective": {"count": 1, "max": 0.004249175175650036, "sum": 0.004249175175650036, "min": 0.004249175175650036}}, "EndTime": 1587184768.394862, "Dimensions": {"model": 1, "Host": "algo-1", "Operation": "training", "Algorithm": "Linear Learner", "epoch": 14}, "StartTime": 1587184768.39484}

#metrics {"Metrics": {"train_binary_classification_cross_entropy_objective": {"count": 1, "max": 0.00486598


2020-04-18 04:39:38 Uploading - Uploading generated training model
2020-04-18 04:39:38 Completed - Training job completed
Training seconds: 139
Billable seconds: 139
CPU times: user 656 ms, sys: 50.9 ms, total: 707 ms
Wall time: 4min 42s


### 部署和评估优化过的评估器

部署并评估优化过的预测器。

我们假设对于经过优化并且提高了召回率的模型，假负例（欺诈性交易错误地被标记为有效交易）的数量应该更少；优化模型后，假负例的数量变少了吗？

In [78]:
%%time 
# deploy and create a predictor
recall_predictor = linear_recall.deploy(initial_instance_count=1, instance_type='ml.t2.medium')

---------------!CPU times: user 251 ms, sys: 19 ms, total: 270 ms
Wall time: 7min 32s


In [79]:
print('Metrics for tuned (recall), LinearLearner.\n')

# get metrics for tuned predictor
metrics = evaluate(recall_predictor, 
                   test_features.astype('float32'), 
                   test_labels, 
                   verbose=True)

Metrics for tuned (recall), LinearLearner.

prediction (col)    0.0   1.0
actual (row)                 
0                 81913  3388
1                    10   131

Recall:     0.929
Precision:  0.037
Accuracy:   0.960



## 删除端点

与之前一样，评估完模型后，应该删除端点。我在下面使用了之前定义的 `delete_endpoint` 辅助函数。

In [80]:
# delete the predictor endpoint 
delete_endpoint(recall_predictor)

Deleted linear-learner-2020-04-18-04-35-18-741


---
## 改进模型：解决类别不平衡性问题

我们的模型经过优化后召回率更高，减少了**假负例**的数量。之前，我们提到类别不平衡性可能会使模型出现偏差，预测所有交易都是有效交易，导致假负例和真负例的数量更多。所以如果解决这种不平衡性，模型将得到进一步优化。

为了在训练二元分类器的过程中考虑类别不平衡性问题，我们可以使用 LinearLearner 的超参数 `positive_example_weight_mult`，它表示在训练二元分类器时分配给正样本（1，欺诈性数据）的权重。负样本（0，有效数据）的权重固定为 1。

### 练习：创建一个设定 `positive_example_weight_mult` 参数的 LinearLearner

除了提高模型的召回率（可以使用 `linear_recall` 作为起点）之外，应该添加一个考虑到类别不平衡性问题的参数。以下内容摘自超参数 `positive_example_weight_mult` 的[参考文档](https://docs.aws.amazon.com/sagemaker/latest/dg/ll_hyperparameters.html)：
> "如果你希望算法选择一个权重，使分类负样本与正样本的误差对训练损失的影响一样，请指定 `balanced`。"

还可以输入特定的浮点值，在这种情形下，正样本的权重应该比负样本的高，因为正样本更少。

In [81]:
# instantiate a LinearLearner

# include params for tuning for higher recall
# *and* account for class imbalance in training data
linear_balanced = LinearLearner(role=role,
                              train_instance_count=1, 
                              train_instance_type='ml.c4.xlarge',
                              predictor_type='binary_classifier',
                              output_path=output_path,
                              sagemaker_session=sagemaker_session,
                              epochs=15,
                              binary_classifier_model_selection_criteria='precision_at_target_recall', # target recall
                              target_recall=0.9,
                              positive_example_weight_mult='balanced')


### 练习：训练平衡评估器

用新的平衡评估器拟合特殊格式的训练数据。

In [82]:
%%time 
# train the estimator on formatted training data
linear_balanced.fit(formatted_train_data)

2020-04-18 04:54:20 Starting - Starting the training job...
2020-04-18 04:54:21 Starting - Launching requested ML instances......
2020-04-18 04:55:22 Starting - Preparing the instances for training...
2020-04-18 04:56:13 Downloading - Downloading input data......
2020-04-18 04:57:15 Training - Training image download completed. Training in progress..Docker entrypoint called with argument(s): train
Running default environment configuration script
[04/18/2020 04:57:18 INFO 140464811779904] Reading default configuration from /opt/amazon/lib/python2.7/site-packages/algorithm/resources/default-input.json: {u'loss_insensitivity': u'0.01', u'epochs': u'15', u'feature_dim': u'auto', u'init_bias': u'0.0', u'lr_scheduler_factor': u'auto', u'num_calibration_samples': u'10000000', u'accuracy_top_k': u'3', u'_num_kv_servers': u'auto', u'use_bias': u'true', u'num_point_for_scaler': u'10000', u'_log_level': u'info', u'quantile': u'0.5', u'bias_lr_mult': u'auto', u'lr_scheduler_step': u'auto', u'init_

[2020-04-18 04:57:31.814] [tensorio] [info] epoch_stats={"data_pipeline": "/opt/ml/input/data/train", "epoch": 5, "duration": 6825, "num_examples": 200, "num_bytes": 33493320}
#metrics {"Metrics": {"train_binary_classification_weighted_cross_entropy_objective": {"count": 1, "max": 0.4400108514526981, "sum": 0.4400108514526981, "min": 0.4400108514526981}}, "EndTime": 1587185851.81434, "Dimensions": {"model": 0, "Host": "algo-1", "Operation": "training", "Algorithm": "Linear Learner", "epoch": 1}, "StartTime": 1587185851.814254}

#metrics {"Metrics": {"train_binary_classification_weighted_cross_entropy_objective": {"count": 1, "max": 0.41052518957703554, "sum": 0.41052518957703554, "min": 0.41052518957703554}}, "EndTime": 1587185851.814428, "Dimensions": {"model": 1, "Host": "algo-1", "Operation": "training", "Algorithm": "Linear Learner", "epoch": 1}, "StartTime": 1587185851.814413}

#metrics {"Metrics": {"train_binary_classification_weighted_cross_entropy_objective": {"count": 1, "max"

[2020-04-18 04:57:37.425] [tensorio] [info] epoch_stats={"data_pipeline": "/opt/ml/input/data/train", "epoch": 7, "duration": 5604, "num_examples": 200, "num_bytes": 33493320}
#metrics {"Metrics": {"train_binary_classification_weighted_cross_entropy_objective": {"count": 1, "max": 0.39636924766655546, "sum": 0.39636924766655546, "min": 0.39636924766655546}}, "EndTime": 1587185857.425602, "Dimensions": {"model": 0, "Host": "algo-1", "Operation": "training", "Algorithm": "Linear Learner", "epoch": 2}, "StartTime": 1587185857.425508}

#metrics {"Metrics": {"train_binary_classification_weighted_cross_entropy_objective": {"count": 1, "max": 0.376585278074945, "sum": 0.376585278074945, "min": 0.376585278074945}}, "EndTime": 1587185857.425699, "Dimensions": {"model": 1, "Host": "algo-1", "Operation": "training", "Algorithm": "Linear Learner", "epoch": 2}, "StartTime": 1587185857.425677}

#metrics {"Metrics": {"train_binary_classification_weighted_cross_entropy_objective": {"count": 1, "max": 

[2020-04-18 04:57:50.201] [tensorio] [info] epoch_stats={"data_pipeline": "/opt/ml/input/data/train", "epoch": 11, "duration": 6351, "num_examples": 200, "num_bytes": 33493320}
#metrics {"Metrics": {"train_binary_classification_weighted_cross_entropy_objective": {"count": 1, "max": 0.3651828865549672, "sum": 0.3651828865549672, "min": 0.3651828865549672}}, "EndTime": 1587185870.201201, "Dimensions": {"model": 0, "Host": "algo-1", "Operation": "training", "Algorithm": "Linear Learner", "epoch": 4}, "StartTime": 1587185870.201115}

#metrics {"Metrics": {"train_binary_classification_weighted_cross_entropy_objective": {"count": 1, "max": 0.3535009060193546, "sum": 0.3535009060193546, "min": 0.3535009060193546}}, "EndTime": 1587185870.201302, "Dimensions": {"model": 1, "Host": "algo-1", "Operation": "training", "Algorithm": "Linear Learner", "epoch": 4}, "StartTime": 1587185870.201281}

#metrics {"Metrics": {"train_binary_classification_weighted_cross_entropy_objective": {"count": 1, "max":

[2020-04-18 04:57:56.809] [tensorio] [info] epoch_stats={"data_pipeline": "/opt/ml/input/data/train", "epoch": 13, "duration": 6601, "num_examples": 200, "num_bytes": 33493320}
#metrics {"Metrics": {"train_binary_classification_weighted_cross_entropy_objective": {"count": 1, "max": 0.35806176627461034, "sum": 0.35806176627461034, "min": 0.35806176627461034}}, "EndTime": 1587185876.809187, "Dimensions": {"model": 0, "Host": "algo-1", "Operation": "training", "Algorithm": "Linear Learner", "epoch": 5}, "StartTime": 1587185876.809086}

#metrics {"Metrics": {"train_binary_classification_weighted_cross_entropy_objective": {"count": 1, "max": 0.34855323484794576, "sum": 0.34855323484794576, "min": 0.34855323484794576}}, "EndTime": 1587185876.809292, "Dimensions": {"model": 1, "Host": "algo-1", "Operation": "training", "Algorithm": "Linear Learner", "epoch": 5}, "StartTime": 1587185876.80927}

#metrics {"Metrics": {"train_binary_classification_weighted_cross_entropy_objective": {"count": 1, "

...

[2020-04-18 04:58:21.924] [tensorio] [info] epoch_stats={"data_pipeline": "/opt/ml/input/data/train", "epoch": 21, "duration": 5936, "num_examples": 200, "num_bytes": 33493320}
#metrics {"Metrics": {"train_binary_classification_weighted_cross_entropy_objective": {"count": 1, "max": 0.34497469671048114, "sum": 0.34497469671048114, "min": 0.34497469671048114}}, "EndTime": 1587185901.924219, "Dimensions": {"model": 0, "Host": "algo-1", "Operation": "training", "Algorithm": "Linear Learner", "epoch": 9}, "StartTime": 1587185901.92412}

#metrics {"Metrics": {"train_binary_classification_weighted_cross_entropy_objective": {"count": 1, "max": 0.33982736052700024, "sum": 0.33982736052700024, "min": 0.33982736052700024}}, "EndTime": 1587185901.924322, "Dimensions": {"model": 1, "Host": "algo-1", "Operation": "training", "Algorithm": "Linear Learner", "epoch": 9}, "StartTime": 1587185901.924301}

#metrics {"Metrics": {"train_binary_classification_weighted_cross_entropy_objective": {"count": 1, "

[2020-04-18 04:58:28.253] [tensorio] [info] epoch_stats={"data_pipeline": "/opt/ml/input/data/train", "epoch": 23, "duration": 6322, "num_examples": 200, "num_bytes": 33493320}
#metrics {"Metrics": {"train_binary_classification_weighted_cross_entropy_objective": {"count": 1, "max": 0.343385628781726, "sum": 0.343385628781726, "min": 0.343385628781726}}, "EndTime": 1587185908.253386, "Dimensions": {"model": 0, "Host": "algo-1", "Operation": "training", "Algorithm": "Linear Learner", "epoch": 10}, "StartTime": 1587185908.253293}

#metrics {"Metrics": {"train_binary_classification_weighted_cross_entropy_objective": {"count": 1, "max": 0.33867450073855604, "sum": 0.33867450073855604, "min": 0.33867450073855604}}, "EndTime": 1587185908.253487, "Dimensions": {"model": 1, "Host": "algo-1", "Operation": "training", "Algorithm": "Linear Learner", "epoch": 10}, "StartTime": 1587185908.253465}

#metrics {"Metrics": {"train_binary_classification_weighted_cross_entropy_objective": {"count": 1, "max

[2020-04-18 04:58:40.986] [tensorio] [info] epoch_stats={"data_pipeline": "/opt/ml/input/data/train", "epoch": 27, "duration": 6288, "num_examples": 200, "num_bytes": 33493320}
#metrics {"Metrics": {"train_binary_classification_weighted_cross_entropy_objective": {"count": 1, "max": 0.34114156809284457, "sum": 0.34114156809284457, "min": 0.34114156809284457}}, "EndTime": 1587185920.98651, "Dimensions": {"model": 0, "Host": "algo-1", "Operation": "training", "Algorithm": "Linear Learner", "epoch": 12}, "StartTime": 1587185920.986413}

#metrics {"Metrics": {"train_binary_classification_weighted_cross_entropy_objective": {"count": 1, "max": 0.3368815071451005, "sum": 0.3368815071451005, "min": 0.3368815071451005}}, "EndTime": 1587185920.986612, "Dimensions": {"model": 1, "Host": "algo-1", "Operation": "training", "Algorithm": "Linear Learner", "epoch": 12}, "StartTime": 1587185920.98659}

#metrics {"Metrics": {"train_binary_classification_weighted_cross_entropy_objective": {"count": 1, "ma

[2020-04-18 04:58:47.237] [tensorio] [info] epoch_stats={"data_pipeline": "/opt/ml/input/data/train", "epoch": 29, "duration": 6244, "num_examples": 200, "num_bytes": 33493320}
#metrics {"Metrics": {"train_binary_classification_weighted_cross_entropy_objective": {"count": 1, "max": 0.3403380548678451, "sum": 0.3403380548678451, "min": 0.3403380548678451}}, "EndTime": 1587185927.237551, "Dimensions": {"model": 0, "Host": "algo-1", "Operation": "training", "Algorithm": "Linear Learner", "epoch": 13}, "StartTime": 1587185927.237453}

#metrics {"Metrics": {"train_binary_classification_weighted_cross_entropy_objective": {"count": 1, "max": 0.3361356635165574, "sum": 0.3361356635165574, "min": 0.3361356635165574}}, "EndTime": 1587185927.237653, "Dimensions": {"model": 1, "Host": "algo-1", "Operation": "training", "Algorithm": "Linear Learner", "epoch": 13}, "StartTime": 1587185927.237633}

#metrics {"Metrics": {"train_binary_classification_weighted_cross_entropy_objective": {"count": 1, "max


2020-04-18 04:59:04 Uploading - Uploading generated training model
2020-04-18 04:59:04 Completed - Training job completed
Training seconds: 171
Billable seconds: 171
CPU times: user 720 ms, sys: 32.6 ms, total: 753 ms
Wall time: 5min 14s


### 练习：部署并评估平衡评估器

部署并评估平衡预测器。结果符合你的预期吗？

In [83]:
%%time 
# deploy and create a predictor
balanced_predictor = linear_balanced.deploy(initial_instance_count=1, instance_type='ml.t2.medium')

---------------!CPU times: user 259 ms, sys: 8.45 ms, total: 267 ms
Wall time: 7min 32s


In [84]:
print('Metrics for balanced, LinearLearner.\n')

# get metrics for balanced predictor
metrics = evaluate(balanced_predictor, 
                   test_features.astype('float32'), 
                   test_labels, 
                   verbose=True)

Metrics for balanced, LinearLearner.

prediction (col)    0.0   1.0
actual (row)                 
0                 84276  1025
1                    12   129

Recall:     0.915
Precision:  0.112
Accuracy:   0.988



## 删除端点

评估完模型后，应该删除端点。

In [85]:
# delete the predictor endpoint 
delete_endpoint(balanced_predictor)

Deleted linear-learner-2020-04-18-04-54-20-427


指标变化注意事项：

上述模型在召回率固定为约 90% 的情况下，经过优化尽量提高精确率。召回率在训练过程中固定为 90%，但是当我们将训练过的模型应用到测试数据集上时，召回率可能会变化。

---
## 模型设计
你已经知道如何优化和平衡 LinearLearner。下面请创建、训练和部署你自己的模型。这道练习属于开放式练习，使你有机会实践设计和部署模型的步骤。

### 练习：根据指定情形训练和部署一个 LinearLearner，并设定相应的超参数。

**情形：**
* 银行要求你构建一个能帮助提供良好用户体验的模型；用户应该最多只有约 15% 的有效交易被标记为欺诈性交易。

这就需要你做出设计决策：对于上述情形，在训练过程中你应该力求优化什么指标（和值）？

模型在训练集上的效果与在测试集上的效果可能有 5-10% 的偏差。例如，如果在训练集上的准确率为 80%，那么在测试集上的准确率约为 70-90%。

最终模型应该考虑类别不平衡性，并且相应地做出优化。

In [ ]:
%%time
# instantiate and train a LinearLearner

# include params for tuning for higher precision
# *and* account for class imbalance in training data


In [ ]:
%%time 
# deploy and evaluate a predictor


In [ ]:
## IMPORTANT
# delete the predictor endpoint after evaluation 


## 最终清理！

* 仔细检查是否删除了所有端点。
* 还建议直接在 AWS 控制台中手动删除 S3 存储桶、模型和端点配置。

你可以在[此文档](https://docs.aws.amazon.com/sagemaker/latest/dg/ex1-cleanup.html)中查看详细的清理说明。

---
# 总结

在此 notebook 中，你学习了如何在 SageMaker 中训练和部署 LinearLearner。此模型非常适合二元分类任务，并且需要作出设计决策和处理训练集中的类别不平衡性问题。

你根据机器学习工作流程的步骤，加载了一些信用卡交易数据，探索了该数据并为训练模型准备好数据。然后根据不同的设计考虑事项训练、部署和评估了多个模型。